<a href="https://colab.research.google.com/github/FatemehArabzadeh/HuggingFaceProject/blob/main/qa2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question answering (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [2]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

You will need to setup git, adapt your email and name in the following cell.

In [3]:
!git config --global user.email "arabzadehfatemeh313@gmail.com"
!git config --global user.name "FatemeArabzadeh"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from datasets import load_dataset

raw_datasets = load_dataset("SajjadAyoubi/persian_qa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/930 [00:00<?, ? examples/s]

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [7]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی خصوصی در ایران و بزرگ‌ترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم‌اکنون محرک بسیاری از صنایع بالادستی و پایین‌دستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده‌است و همچنین این شرکت در سال ۱۳۹۱ برای نخستین‌بار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ‌ترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده‌است. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده‌رود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم‌آبی زاینده‌رود شناخته می‌شود.
Question:  شرکت فولاد مبارکه در کجا واقع شده است
Answer:  {'text': ['در شرق شهر مبارکه'], 'answer_start': [114]}


In [8]:
raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Filter:   0%|          | 0/9008 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 2702
})

In [9]:
print(raw_datasets["validation"][0]["answers"])
print(raw_datasets["validation"][2]["answers"])

{'text': ['مادرید', 'مادرید'], 'answer_start': [19, 19]}
{'text': ['۱۳', '۱۳'], 'answer_start': [329, 329]}


In [10]:
print(raw_datasets["validation"][2]["context"])
print(raw_datasets["validation"][2]["question"])

باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق‌ترین تیم تاریخ فوتبال اسپانیا و موفق‌ترین تیم تاریخ فوتبال اروپا و موفق‌ترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن‌ها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه‌های جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی است. همچنین رئال مادرید برنده بهترین باشگاه قرن از سوی فیفا شده‌است. همچنین کلمهٔ رئال در زبان اسپانیایی، به معنی «سلطنتی» است، این لقب را شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد. همچنین در همان سال، شکل یک تاج نیز بر روی آرم این باشگاه قرار گرفت. رئال مادرید سال‌هاست که در لالیگا، از تیم‌های قدرتمند محسوب می‌شود و رقیب اصلی این تیم نیز، بارسلونا است. از نظر درآمد، رئال مادرید با ۵۵۷ میلیون یورو درآمد در فصل ۱۵–۲۰۱۴ در صدر پردرآمدترین باشگاه‌های فوتبال جها

In [11]:
from transformers import AutoTokenizer

model_checkpoint = "google-bert/bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [12]:
tokenizer.is_fast

True

In [13]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] شرکت فولاد مبارکه در کجا واقع شده است [SEP] شرکت فولاد مبارکە اصفهان ، بزرگترین واحد صنعتی خصوصی در ایران و بزرگترین مجتمع تولید فولاد در کشور ایران است ، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هماکنون محرک بسیاری از صنایع بالادستی و پاییندستی است. فولاد مبارکه در ۱۱ دوره جایزە ملی تعالی سازمانی و ۶ دوره جایزە شرکت دانشی در کشور رتبە نخست را بدست اوردهاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستینبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزە ملی تعالی سازمانی را از ان خود کند. شرکت فولاد مبارکە اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شدهاست. مصرف اب این کارخانه در کمترین میزان خود ، ۱ ٫ ۵ ٪ از دبی زایندهرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کمابی زایندهرود شناخته میشود. [SEP]'

In [14]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] شرکت فولاد مبارکه در کجا واقع شده است [SEP] شرکت فولاد مبارکە اصفهان ، بزرگترین واحد صنعتی خصوصی در ایران و بزرگترین مجتمع تولید فولاد در کشور ایران است ، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هماکنون محرک بسیاری از صنایع بالادستی و پاییندستی است. فولاد مبارکه در ۱۱ دوره جایزە ملی تعالی سازمانی و [SEP]
[CLS] شرکت فولاد مبارکه در کجا واقع شده است [SEP]بارکه قرار دارد. فولاد مبارکه هماکنون محرک بسیاری از صنایع بالادستی و پاییندستی است. فولاد مبارکه در ۱۱ دوره جایزە ملی تعالی سازمانی و ۶ دوره جایزە شرکت دانشی در کشور رتبە نخست را بدست اوردهاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستینبار به عنوان تنها شرکت ایرانی [SEP]
[CLS] شرکت فولاد مبارکه در کجا واقع شده است [SEP] مبارکه در ۱۱ دوره جایزە ملی تعالی سازمانی و ۶ دوره جایزە شرکت دانشی در کشور رتبە نخست را بدست اوردهاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستینبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزە ملی تعالی سازمانی را از ان خود کند. شرکت فولاد مبارک [SEP]
[CLS] شرکت فولاد مبارکه در کجا واقع شده است [

In [15]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [17]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0, 0, 0, 0]

In [18]:
inputs = tokenizer(
    raw_datasets["train"][2:6]["question"],
    raw_datasets["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 26 features.
Here is where each comes from: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3].


In [20]:
answers = raw_datasets["train"][2:6]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([0,
  0,
  81,
  50,
  19,
  0,
  0,
  13,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  71,
  37,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  52],
 [0,
  0,
  94,
  63,
  32,
  0,
  0,
  20,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  72,
  38,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  82])

In [21]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: تندیس زرین جایزۀ ملی تعالی سازمانی, labels give: [CLS]


In [22]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: تندیس زرین جایزۀ ملی تعالی سازمانی, decoded example: [CLS] شرکت فولاد مبارکه در سال ۱۳۹۱ چه جایزه ای برد ؟ [SEP]۵۴ تندیس زرین جایزە ملی تعالی سازمانی را از ان خود کند. شرکت فولاد مبارکە اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر [SEP]


In [23]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        if len(answer["answer_start"]) > 0 :
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
        else:
            start_char=0
            end_char=0

        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [24]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/9008 [00:00<?, ? examples/s]

(9008, 10192)

In [25]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [26]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

(930, 972)

In [27]:
small_eval_set = raw_datasets["validation"].select(range(100))
trained_checkpoint = "google-bert/bert-base-multilingual-uncased"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [28]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [29]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [31]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [32]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [33]:
import evaluate

metric = evaluate.load("squad")

In [34]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [48]:
predicted_answers = [{"id": str(pred["id"]), "prediction_text": pred["prediction_text"]} for pred in predicted_answers]

# Filter out predictions that do not have a corresponding reference
predicted_answers = [
    pred for pred in predicted_answers if any(ref["id"] == pred["id"] for ref in theoretical_answers)
]
print(predicted_answers[0])
print(theoretical_answers[0])

{'id': '9101', 'prediction_text': 'قیب اصلی این تیم نیز، بارسلونا است. از نظر درآمد، رئال مادرید با ۵۵۷ میلیون ی'}
{'id': '9101', 'answers': {'text': ['مادرید', 'مادرید'], 'answer_start': [19, 19]}}


In [49]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 0.0, 'f1': 2.617423588735339}

In [56]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = str(example["id"])
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [
        {
            "id": str(ex["id"]),  # Ensure ID is a string
            "answers": {
                "text": ex["answers"].get("text", []),  # Ensure 'text' is a list
                "answer_start": ex["answers"].get("answer_start", [])  # Ensure 'answer_start' is a list
            }
        }
        for ex in examples
    ]

    # Convert IDs to strings if they are not already
    predicted_answers = [{"id": str(pred["id"]), "prediction_text": pred["prediction_text"]} for pred in predicted_answers]

    # Compute the metric
    try:
        results = metric.compute(predictions=predicted_answers, references=theoretical_answers)
    except ValueError as e:
        print(f"Error during metric computation: {e}")
        results = {"error": str(e)}

    return results

In [57]:
compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

  0%|          | 0/100 [00:00<?, ?it/s]

Error during metric computation: max() arg is an empty sequence


{'error': 'max() arg is an empty sequence'}

In [58]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
from huggingface_hub import notebook_login

notebook_login()

In [61]:
from transformers import TrainingArguments

args = TrainingArguments(
    "google-bert/bert-base-multilingual-uncased",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [62]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

Step,Training Loss
500,2.913500
1000,2.195300
1500,1.807100
2000,1.539500
2500,1.483600
3000,1.112100
3500,1.057500


TrainOutput(global_step=3822, training_loss=1.6737838497591044, metrics={'train_runtime': 890.329, 'train_samples_per_second': 34.342, 'train_steps_per_second': 4.293, 'total_flos': 5992057225469952.0, 'train_loss': 1.6737838497591044, 'epoch': 3.0})

In [63]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/930 [00:00<?, ?it/s]

Error during metric computation: max() arg is an empty sequence


{'error': 'max() arg is an empty sequence'}

In [64]:
trainer.push_to_hub(commit_message="Training complete")

events.out.tfevents.1723459658.140b58a40f5f.227.0:   0%|          | 0.00/6.91k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Farabzadeh/bert-base-multilingual-uncased/commit/fae6a02c4195c5ddf90e1e16ac94b81aabc623e6', commit_message='Training complete', commit_description='', oid='fae6a02c4195c5ddf90e1e16ac94b81aabc623e6', pr_url=None, pr_revision=None, pr_num=None)

In [65]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [66]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [67]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [70]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [71]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [72]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "bert-base-multilingual-uncased"
repo_name = get_full_repo_name(model_name)
repo_name

'Farabzadeh/bert-base-multilingual-uncased'

In [73]:
output_dir = "bert-base-multilingual-uncased"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Farabzadeh/bert-base-multilingual-uncased into local empty directory.


Download file model.safetensors:   0%|          | 8.00k/636M [00:00<?, ?B/s]

Download file runs/Aug12_10-47-05_140b58a40f5f/events.out.tfevents.1723459658.140b58a40f5f.227.0: 100%|#######…

Clean file runs/Aug12_10-47-05_140b58a40f5f/events.out.tfevents.1723459658.140b58a40f5f.227.0:  15%|#4        …

Download file training_args.bin: 100%|##########| 5.05k/5.05k [00:00<?, ?B/s]

Clean file training_args.bin:  20%|#9        | 1.00k/5.05k [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/636M [00:00<?, ?B/s]

In [74]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )
    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/3822 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/930 [00:00<?, ?it/s]

Error during metric computation: max() arg is an empty sequence
epoch 0: {'error': 'max() arg is an empty sequence'}
Evaluation!


...

  0%|          | 0/930 [00:00<?, ?it/s]

Error during metric computation: max() arg is an empty sequence
epoch 1: {'error': 'max() arg is an empty sequence'}
Evaluation!


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/930 [00:00<?, ?it/s]

Error during metric computation: max() arg is an empty sequence
epoch 2: {'error': 'max() arg is an empty sequence'}


In [75]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [77]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "Farabzadeh/bert-base-multilingual-uncased"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
🤗 انسان های اولیه در مکان های زیادی زندگی می کردند  . این مکان ها شامل غار ، دشت ، کوه و ... می باشد آن ها همچنین از غذا هایی مثل گوشت حیوانات و پرندگان همچنین گیاهان استفاده می کردند و به تفریح می پرداختند.
"""
question = "انسان های اولیه چه چیزهایی می خوردند؟"
question_answerer(question=question, context=context)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.35455122590065,
 'start': 131,
 'end': 167,
 'answer': 'گوشت حیوانات و پرندگان همچنین گیاهان'}